<a href="https://colab.research.google.com/github/dpostolovski/eeg_is/blob/train_compare_full_data/Phase%204%20-%20Working%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [108]:
#@title Инсталирање и вчитување на потребните библиотеки
!pip install mne 
!pip install termcolor
!wget "https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/EEGModels.py"

%tensorflow_version 1.12.0

import os
import numpy as np
import mne

from sklearn import metrics 
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

from scipy.io import loadmat
from EEGModels import DeepConvNet
from collections import Counter

K.set_image_data_format('channels_first')

!mkdir saved_models

--2020-06-14 04:06:57--  https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/EEGModels.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18283 (18K) [text/plain]
Saving to: ‘EEGModels.py.1’

EEGModels.py.1      100%[===================>]  17.85K  --.-KB/s    in 0.004s  

2020-06-14 04:06:58 (4.65 MB/s) - ‘EEGModels.py.1’ saved [18283/18283]

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.12.0`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
mkdir: cannot create directory ‘saved_models’: File exists


In [0]:
def reshape(data, labels, events, targets):
  mne_array = np.swapaxes(data, 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0], 1, 8, 350)

  labels_arr = labels.astype(np.int)
  events_arr = events.astype(np.int)
  return mne_array, labels_arr-1, events_arr-1

In [0]:
def reshape_data_to_mne_format(data):
  mne_array = np.swapaxes(data, 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0], 1, 8, 350)

  return mne_array

In [139]:
# Иницијализација на променливите каде ќе бидат вчитани податоците
train_data = []
train_labels = []
train_events = []
train_targets = []
models = []

for i in range(1, 3): # Итерација низ секој испитен примерок
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")

  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_train_data = np.empty(0)
  temp_train_labels = np.empty(0)
  temp_train_events = np.empty(0)
  temp_train_targets = np.empty(0)

  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') + '/Train'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_train_data.size != 0:
      temp_train_data = np.concatenate((temp_train_data, temp), axis=2)
    else: 
      temp_train_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_train_labels.size != 0:
        temp_train_labels = np.concatenate((temp_train_labels, temp))
      else:
        temp_train_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_train_events.size != 0:
        temp_train_events = np.append(temp_train_events, temp)
      else:
        temp_train_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_train_targets.size != 0:
        temp_train_targets = np.concatenate((temp_train_targets, temp))
      else:
        temp_train_targets = np.array(temp)

    print("\t - Податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  train_data.append(temp_train_data)
  train_labels.append(temp_train_labels)
  train_events.append(temp_train_events)
  train_targets.append(temp_train_targets)
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.\n")
      

  data, labels, events = reshape(train_data[i-1], train_labels[i-1], train_events[i-1], train_targets[i-1])
  X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)

  model = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model.evaluate(X_test, y_test, verbose=1)
  print(score)

  models.append(model) # Ke imame 15 modeli neli, pa vo niza se staveni
                       # pristap do niv model[i]

Вчитување тест податоци од испитниот примерок 1...
	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 1 се вчитани.

Train on 3600 samples, validate on 1200 samples
Epoch 1/150
3500/3600 [============================>.] - ETA: 0s - loss: 2.2587 - acc: 0.1614
Epoch 00001: val_loss improved from inf to 2.19180, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 13s 4ms/sample - loss: 2.2597 - acc: 0.1614 - val_loss: 2.1918 - val_acc: 0.1133
Epoch 2/150
3500/3600 [============================>.] - ETA: 0s - loss: 2.1449 - acc: 0.1849
Epoch 00002: val_loss did not improve from 2.19180
3600/3600 [==============================] - 1s 199us/sample - loss: 2.1521 - acc: 0.1836 - val_loss: 2.3058 - val_acc: 0.1150
Epoch 3/150
3400/3600 [===========================>..] - ETA: 0s - loss: 2.1121 - acc: 0.1926
Epoch 00003: val_loss did not improve fro

In [147]:
# Иницијализација на променливите каде ќе бидат вчитани тест податоците
test_data = []
test_events = []
test_runs_per_block = [[i for i in range(3)] for j in range(15)] # Covek, Sesija

for i in range(1, 3): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")
      
  
  # =========================================================================

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = models[i-1].predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()

      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (1) ======================
Вчитување тест податоци од испитниот примерок 1...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 1 се вчитани.

SBJ00| Test_data: (8, 350, 8000)
SBJ00| Test_events: 8000
SBJ00 / S00| Runs per block: 6
SBJ00 / S01| Runs per block: 7
SBJ00 / S02| Runs per block: 7
SBJ00| Predictions: 8000
============== Сесија (0) ==============
5,5,2,5,5,2,2,5,5,5,5,3,6,4,3,5,3,2,2,0,6,7,0,7,6,3,7,3,7,5,2,5,5,3,2,6,6,2,4,2,2,7,3,3,7,7,3,4,4,6,
============== Сесија (1) ==============
5,2,1,0,0,0,2,5,1,1,5,1,6,2,1,2,0,0,2,0,2,2,0,0,2,0,2,5,0,2,0,2,2,5,1,3,1,4,5,1,2,5,5,0,0,1,2,5,5,0,
============== Сесија (2) ==============
0,2,6,6,6,2,2,2,2,6,6,2,2,6,5,2,2,2,2,2,2,2,6,0,5,7,2,5,0,7,2,2,5,5,5,2,5,2,5,2,2,2,6,2,5,0,2,6,0,5,
Stigna li do kraj: True
====================== Примерок (1) ======================


==================

In [0]:
# test_data = reshape_data_to_mne_format(test_data)
# print(test_data.shape)
# predictions = model.predict(X_test)
# np.savetxt("predictions.csv", predictions, delimiter=",")
# np.savetxt("y_test.csv", y_test, delimiter=",")


# predictions = model.predict(X_test)
# np.savetxt("predictions.csv", predictions, delimiter=",")
# np.savetxt("y_test.csv", y_test, delimiter=",")

# from collections import Counter
# int_pred = np.argmax(predictions, axis=1)
# int_ytest = np.argmax(y_test, axis=1)

# correct = 0
# for i in range(20):
#   block_y = int_ytest[i*80:(i+1)*80]
#   block_y_pred = int_pred[i*80:(i+1)*80]

#   class_y = np.bincount(block_y).argmax()
#   class_y_pred = np.bincount(block_y_pred).argmax()

#   print(f"Class Y: {class_y}, prediciton: {class_y_pred}")
#   if class_y == class_y_pred:
#     correct = correct+1

# correct